# Marketing Mix Modelling (MMM) - Multiplicative Model

## Companion to Chapter 9b: Additive vs Multiplicative Models

This notebook implements **multiplicative MMM specifications** as an alternative to the additive approach in the main notebook.

### Key Differences

| Aspect | Additive | Multiplicative |
|--------|----------|----------------|
| Formula | `Y = B + b1*X1 + b2*X2` | `Y = B * (1+L1) * (1+L2)` or `log(Y) = a + b1*log(X1)` |
| Coefficients | Absolute dollar contribution | Elasticities (% change) |
| Interactions | Must add explicitly | Natural/built-in |
| Zero handling | Works fine | Requires transformation |
| Decomposition | Simple sum | Shapley values needed |

### When to Use Multiplicative

- Channels genuinely interact (TV + Search synergy)
- Business thinks in percentage lifts
- You're comfortable with Shapley decomposition

### This Notebook Covers

1. **Log-Log Specification**: Elasticity-based model
2. **Lift-Factor Specification**: Multiplicative lifts from baseline
3. **Shapley Value Decomposition**: Fair attribution for multiplicative models
4. **Comparison with Additive**: Same data, different approach

---
## Setup and Imports

In [ ]:
# Core data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Bayesian modelling
import pymc as pm
import arviz as az

# Metrics
from sklearn.metrics import r2_score, mean_absolute_percentage_error

# For Shapley values
from itertools import combinations
from math import factorial

# Settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

%config InlineBackend.figure_format = 'retina'

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print(f"PyMC version: {pm.__version__}")
print(f"ArviZ version: {az.__version__}")

---
## Part I: Data Preparation

We use the same dataset and preprocessing as the additive notebook.

In [ ]:
# Load the dataset
df_raw = pd.read_csv('conjura_mmm_data.csv')

print(f"Dataset shape: {df_raw.shape}")
print(f"Date range: {df_raw['DATE_DAY'].min()} to {df_raw['DATE_DAY'].max()}")

In [ ]:
# Find organisation with good data coverage
spend_cols = [col for col in df_raw.columns if 'SPEND' in col]

org_stats = df_raw.groupby('ORGANISATION_ID').agg({
    'ALL_PURCHASES': 'sum',
    'DATE_DAY': 'count',
    **{col: lambda x: (x > 0).sum() for col in spend_cols}
}).reset_index()

org_stats.columns = ['ORGANISATION_ID', 'total_purchases', 'n_days'] + [f'{col}_days' for col in spend_cols]

# Select organisation with highest purchases and good coverage
selected_org = org_stats.nlargest(5, 'total_purchases').iloc[0]['ORGANISATION_ID']
df_org = df_raw[df_raw['ORGANISATION_ID'] == selected_org].copy()

print(f"Selected organisation: {selected_org}")
print(f"Records: {len(df_org)}")
print(f"Total purchases: {df_org['ALL_PURCHASES'].sum():,.0f}")

In [ ]:
# Aggregate to weekly data
df_org['DATE_DAY'] = pd.to_datetime(df_org['DATE_DAY'])
df_org['week'] = df_org['DATE_DAY'].dt.to_period('W').dt.start_time

# Columns to aggregate
numeric_cols = df_org.select_dtypes(include=[np.number]).columns.tolist()
cols_to_sum = [col for col in numeric_cols if col not in ['MMM_TIMESERIES_ID']]

df_weekly = df_org.groupby('week')[cols_to_sum].sum().reset_index()
df_weekly = df_weekly.rename(columns={'week': 'date'})

print(f"Weekly data shape: {df_weekly.shape}")
print(f"Weeks: {len(df_weekly)}")

In [ ]:
# Select media channels with meaningful spend
spend_cols = [col for col in df_weekly.columns if 'SPEND' in col]
channel_totals = df_weekly[spend_cols].sum()
active_channels = channel_totals[channel_totals > channel_totals.max() * 0.05].index.tolist()

print("Active media channels (>5% of max spend):")
for ch in active_channels:
    print(f"  {ch}: ${channel_totals[ch]:,.0f}")

media_channels = active_channels

In [ ]:
# Create working dataframe
df = df_weekly[['date', 'ALL_PURCHASES'] + media_channels].copy()

# Create trend variable
df['trend'] = np.arange(len(df))

# Create Fourier features for seasonality
def create_fourier_features(n_periods, period=52, n_harmonics=3):
    """Create Fourier features for seasonality."""
    t = np.arange(n_periods)
    features = []
    names = []
    for k in range(1, n_harmonics + 1):
        features.append(np.sin(2 * np.pi * k * t / period))
        features.append(np.cos(2 * np.pi * k * t / period))
        names.extend([f'sin_{k}', f'cos_{k}'])
    return np.column_stack(features), names

X_fourier, fourier_names = create_fourier_features(len(df), period=52, n_harmonics=3)
for i, name in enumerate(fourier_names):
    df[name] = X_fourier[:, i]

print(f"Working dataframe shape: {df.shape}")
df.head()

---
## Part II: Transformation Functions

Same adstock and saturation functions as the additive model.

In [ ]:
def geometric_adstock(x, decay_rate):
    """
    Apply geometric adstock transformation.
    
    Adstock(t) = Spend(t) + decay_rate * Adstock(t-1)
    
    Parameters:
    -----------
    x : array-like
        Raw spend values
    decay_rate : float
        Decay parameter (0-1). Higher = longer carryover.
    
    Returns:
    --------
    array : Adstocked values
    """
    x = np.asarray(x, dtype=np.float64)
    adstocked = np.zeros_like(x)
    adstocked[0] = x[0]
    for t in range(1, len(x)):
        adstocked[t] = x[t] + decay_rate * adstocked[t-1]
    return adstocked


def hill_function(x, K, S):
    """
    Apply Hill saturation function.
    
    Response = x^S / (K^S + x^S)
    
    Parameters:
    -----------
    x : array-like
        Adstocked spend
    K : float
        Half-saturation point (spend at 50% of max effect)
    S : float
        Shape parameter (steepness of curve)
    
    Returns:
    --------
    array : Saturated values (0-1 range)
    """
    x = np.asarray(x, dtype=np.float64)
    return np.power(x, S) / (np.power(K, S) + np.power(x, S) + 1e-10)


print("Transformation functions defined.")

---
## Part III: Prepare Data for Multiplicative Models

Key difference: We need to handle zeros carefully for log transformations.

In [ ]:
# Target variable
target_col = 'ALL_PURCHASES'
y_raw = df[target_col].values.astype(np.float64)

# Check for zeros in target (problematic for log)
print(f"Target zeros: {(y_raw == 0).sum()}")
print(f"Target min: {y_raw.min():.2f}")
print(f"Target mean: {y_raw.mean():.2f}")

# Scale target (for additive comparison later)
y_mean = y_raw.mean()
y_scaled = y_raw / y_mean

# Log transform for multiplicative (use log1p for safety)
y_log = np.log(y_raw + 1)  # log(y + 1) to handle any zeros

print(f"\nLog-transformed target range: [{y_log.min():.2f}, {y_log.max():.2f}]")

In [ ]:
# Prepare media data with adstock pre-applied
n_obs = len(df)
n_channels = len(media_channels)

# Pre-compute adstocked spend for each channel
# Using moderate decay rates as starting point
default_decay = 0.5
X_media_raw = np.zeros((n_obs, n_channels), dtype=np.float64)
X_media_adstocked = np.zeros((n_obs, n_channels), dtype=np.float64)

spend_means = {}

for i, channel in enumerate(media_channels):
    raw_spend = df[channel].values.astype(np.float64)
    X_media_raw[:, i] = raw_spend
    
    # Apply adstock
    adstocked = geometric_adstock(raw_spend, default_decay)
    
    # Scale by mean for numerical stability
    mean_val = adstocked.mean() if adstocked.mean() > 0 else 1.0
    spend_means[channel] = mean_val
    X_media_adstocked[:, i] = adstocked / mean_val
    
    print(f"{channel}: mean adstocked = {mean_val:,.0f}, zeros = {(raw_spend == 0).sum()}")

# For log-log model, we need log(1 + x) to handle zeros
X_media_log = np.log1p(X_media_adstocked)

In [ ]:
# Prepare other variables
trend = df['trend'].values.astype(np.float64)
trend_scaled = trend / trend.max()  # Scale to [0, 1]

X_fourier = df[fourier_names].values.astype(np.float64)

print(f"\nData shapes:")
print(f"  y_log: {y_log.shape}")
print(f"  X_media_log: {X_media_log.shape}")
print(f"  X_fourier: {X_fourier.shape}")
print(f"  trend_scaled: {trend_scaled.shape}")

---
## Part IV: Model 1 - Log-Log Multiplicative Model

### The Specification

```
log(Y) = alpha + beta_trend * trend + beta_fourier * fourier + sum(beta_i * log(1 + X_i)) + epsilon
```

In this model:
- **beta_i** represents the **elasticity** of channel i
- Interpretation: A 1% increase in spend leads to a beta_i% increase in sales
- Typical elasticities are small (0.01 - 0.30)

In [ ]:
# Build Log-Log Multiplicative Model

with pm.Model() as log_log_model:
    
    # ============= BASELINE & CONTROLS =============
    
    # Intercept (in log space)
    alpha = pm.Normal('alpha', mu=np.log(y_raw.mean()), sigma=1)
    
    # Trend coefficient (small effect expected)
    beta_trend = pm.Normal('beta_trend', mu=0, sigma=0.5)
    
    # Seasonality coefficients (Fourier)
    n_fourier = X_fourier.shape[1]
    beta_fourier = pm.Normal('beta_fourier', mu=0, sigma=0.3, shape=n_fourier)
    
    # ============= MEDIA ELASTICITIES =============
    # Elasticities are typically positive and small (0.01 to 0.3)
    # Using HalfNormal to enforce positive effect
    
    beta_media = pm.HalfNormal('beta_media', sigma=0.15, shape=n_channels)
    
    # ============= NOISE =============
    
    sigma = pm.HalfNormal('sigma', sigma=0.5)
    
    # ============= BUILD LOG(MU) =============
    
    # Baseline (log scale)
    log_mu = alpha
    
    # Trend component
    log_mu = log_mu + beta_trend * trend_scaled
    
    # Seasonality component
    log_mu = log_mu + pm.math.dot(X_fourier, beta_fourier)
    
    # Media effects (elasticities * log spend)
    for i in range(n_channels):
        log_mu = log_mu + beta_media[i] * X_media_log[:, i]
    
    # ============= LIKELIHOOD =============
    # LogNormal distribution for the response
    # This is equivalent to: log(Y) ~ Normal(log_mu, sigma)
    
    y_obs = pm.Normal('y_obs', mu=log_mu, sigma=sigma, observed=y_log)

print("Log-Log Multiplicative Model specification complete!")
print("\nModel interprets coefficients as ELASTICITIES:")
print("  beta = 0.10 means: 1% more spend -> 0.10% more sales")
print(log_log_model)

In [ ]:
# Fit the Log-Log model
with log_log_model:
    trace_loglog = pm.sample(
        draws=2000,
        tune=1000,
        chains=4,
        cores=4,
        target_accept=0.9,
        random_seed=RANDOM_SEED,
        return_inferencedata=True
    )

In [ ]:
# Check convergence
print("=" * 60)
print("LOG-LOG MODEL CONVERGENCE DIAGNOSTICS")
print("=" * 60)

summary_loglog = az.summary(trace_loglog, var_names=['alpha', 'beta_trend', 'beta_media', 'sigma'])
print(summary_loglog)

# Check R-hat
r_hat_ok = (summary_loglog['r_hat'] < 1.01).all()
print(f"\nAll R-hat < 1.01: {r_hat_ok}")

In [ ]:
# Trace plots for Log-Log model
az.plot_trace(trace_loglog, var_names=['beta_media'], figsize=(12, 2*n_channels))
plt.suptitle('Log-Log Model: Media Elasticities Trace Plots', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Extract and interpret elasticities
elasticities = trace_loglog.posterior['beta_media'].mean(dim=['chain', 'draw']).values

print("\n" + "=" * 60)
print("MEDIA ELASTICITIES (Log-Log Model)")
print("=" * 60)
print("\nInterpretation: 1% increase in spend -> X% increase in sales\n")

for i, channel in enumerate(media_channels):
    el = elasticities[i]
    ci_low = np.percentile(trace_loglog.posterior['beta_media'].values[:, :, i], 2.5)
    ci_high = np.percentile(trace_loglog.posterior['beta_media'].values[:, :, i], 97.5)
    print(f"{channel}:")
    print(f"  Elasticity: {el:.4f} [{ci_low:.4f}, {ci_high:.4f}]")
    print(f"  Meaning: 10% more spend -> {el*10:.2f}% more sales")
    print()

---
## Part V: Model 2 - Lift-Factor Multiplicative Model

### The Specification

```
Y = Baseline * (1 + lift_TV) * (1 + lift_Digital) * ... * seasonality * exp(noise)
```

Where each lift is:
```
lift_i = gamma_i * saturate(adstock(spend_i))
```

This model:
- Naturally captures channel interactions
- Predictions are always positive
- Baseline represents true "no marketing" sales

In [ ]:
# Build Lift-Factor Multiplicative Model

with pm.Model() as lift_model:
    
    # ============= BASELINE =============
    # Baseline sales with no marketing (log scale for positivity)
    log_baseline = pm.Normal('log_baseline', mu=np.log(y_raw.mean() * 0.7), sigma=0.5)
    baseline = pm.Deterministic('baseline', pm.math.exp(log_baseline))
    
    # ============= TREND MULTIPLIER =============
    # Trend as a multiplicative factor
    trend_coef = pm.Normal('trend_coef', mu=0, sigma=0.3)
    trend_multiplier = 1 + trend_coef * trend_scaled
    
    # ============= SEASONALITY MULTIPLIER =============
    n_fourier = X_fourier.shape[1]
    beta_fourier = pm.Normal('beta_fourier', mu=0, sigma=0.2, shape=n_fourier)
    seasonality_effect = pm.math.dot(X_fourier, beta_fourier)
    seasonality_multiplier = 1 + seasonality_effect
    
    # ============= MEDIA LIFT FACTORS =============
    # Gamma controls the maximum possible lift from each channel
    # E.g., gamma=0.3 means channel can lift sales by up to 30%
    gamma = pm.HalfNormal('gamma', sigma=0.3, shape=n_channels)
    
    # Saturation parameters (shared Hill function approach)
    K = pm.LogNormal('K', mu=0, sigma=0.5, shape=n_channels)
    S_raw = pm.Beta('S_raw', alpha=2, beta=2, shape=n_channels)
    S = pm.Deterministic('S', 0.3 + 1.2 * S_raw)
    
    # ============= COMPUTE TOTAL LIFT =============
    # Start with multiplier of 1
    total_multiplier = trend_multiplier * seasonality_multiplier
    
    # Apply each channel's lift
    for i in range(n_channels):
        # Saturated media effect (0 to 1 range)
        saturated = (X_media_adstocked[:, i] ** S[i]) / (K[i] ** S[i] + X_media_adstocked[:, i] ** S[i] + 1e-10)
        
        # Lift factor: 1 + gamma * saturated
        channel_lift = 1 + gamma[i] * saturated
        
        # Multiply into total (this creates natural interactions!)
        total_multiplier = total_multiplier * channel_lift
    
    # ============= EXPECTED VALUE =============
    mu = baseline * total_multiplier
    
    # ============= NOISE =============
    sigma = pm.HalfNormal('sigma', sigma=0.3)
    
    # ============= LIKELIHOOD =============
    # LogNormal ensures positive predictions
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma * y_raw.mean(), observed=y_raw)

print("Lift-Factor Multiplicative Model specification complete!")
print("\nModel interprets gamma as MAXIMUM LIFT POTENTIAL:")
print("  gamma = 0.20 means: channel can lift sales by up to 20%")
print(lift_model)

In [ ]:
# Fit the Lift-Factor model
with lift_model:
    trace_lift = pm.sample(
        draws=2000,
        tune=1500,
        chains=4,
        cores=4,
        target_accept=0.95,  # Higher for multiplicative model stability
        random_seed=RANDOM_SEED,
        return_inferencedata=True
    )

In [ ]:
# Check convergence
print("=" * 60)
print("LIFT-FACTOR MODEL CONVERGENCE DIAGNOSTICS")
print("=" * 60)

summary_lift = az.summary(trace_lift, var_names=['baseline', 'gamma', 'K', 'S', 'sigma'])
print(summary_lift)

# Check R-hat
r_hat_ok = (summary_lift['r_hat'] < 1.05).all()
print(f"\nAll R-hat < 1.05: {r_hat_ok}")

In [ ]:
# Trace plots for Lift model
az.plot_trace(trace_lift, var_names=['gamma', 'baseline'], figsize=(12, 2*(n_channels+1)))
plt.suptitle('Lift-Factor Model: Gamma (Max Lift) Trace Plots', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Extract and interpret lift factors
gammas = trace_lift.posterior['gamma'].mean(dim=['chain', 'draw']).values
baseline_est = trace_lift.posterior['baseline'].mean(dim=['chain', 'draw']).values

print("\n" + "=" * 60)
print("LIFT FACTORS (Multiplicative Model)")
print("=" * 60)
print(f"\nBaseline (no marketing): {baseline_est:,.0f} sales/week")
print(f"Average actual sales: {y_raw.mean():,.0f} sales/week")
print(f"Implied total lift: {(y_raw.mean() / baseline_est - 1)*100:.1f}%")
print("\nChannel Maximum Lift Potential:\n")

for i, channel in enumerate(media_channels):
    g = gammas[i]
    ci_low = np.percentile(trace_lift.posterior['gamma'].values[:, :, i], 2.5)
    ci_high = np.percentile(trace_lift.posterior['gamma'].values[:, :, i], 97.5)
    print(f"{channel}:")
    print(f"  Max lift (gamma): {g:.3f} [{ci_low:.3f}, {ci_high:.3f}]")
    print(f"  Meaning: At saturation, lifts sales by up to {g*100:.1f}%")
    print()

---
## Part VI: Shapley Value Decomposition

In multiplicative models, simple subtraction doesn't work for attribution.
Shapley values provide a **fair** decomposition:

- Each channel gets credit based on its **marginal contribution** across all possible orderings
- The sum of Shapley values equals the total effect
- Satisfies fairness axioms (symmetry, efficiency, linearity)

In [ ]:
def compute_shapley_values(baseline, channel_lifts):
    """
    Compute Shapley values for multiplicative attribution.
    
    Parameters:
    -----------
    baseline : float
        Baseline sales (no marketing)
    channel_lifts : dict
        Dictionary of {channel_name: lift_array} where lift_array
        contains the multiplicative lift (1 + effect) for each time period
    
    Returns:
    --------
    dict : Shapley value contribution for each channel
    """
    channels = list(channel_lifts.keys())
    n = len(channels)
    n_obs = len(list(channel_lifts.values())[0])
    
    # Initialize Shapley values
    shapley = {ch: np.zeros(n_obs) for ch in channels}
    
    # For each channel, compute its marginal contribution across all coalitions
    for i, channel in enumerate(channels):
        other_channels = [c for c in channels if c != channel]
        
        # Iterate over all possible subsets of other channels
        for size in range(len(other_channels) + 1):
            for subset in combinations(other_channels, size):
                # Coalition without the channel
                value_without = baseline.copy() if isinstance(baseline, np.ndarray) else np.full(n_obs, baseline)
                for c in subset:
                    value_without = value_without * channel_lifts[c]
                
                # Coalition with the channel
                value_with = value_without * channel_lifts[channel]
                
                # Marginal contribution
                marginal = value_with - value_without
                
                # Shapley weight
                weight = (factorial(size) * factorial(n - size - 1)) / factorial(n)
                
                shapley[channel] += weight * marginal
    
    return shapley

print("Shapley value function defined.")

In [ ]:
# Compute Shapley values for the Lift-Factor model

# Extract posterior means
gamma_means = trace_lift.posterior['gamma'].mean(dim=['chain', 'draw']).values
K_means = trace_lift.posterior['K'].mean(dim=['chain', 'draw']).values
S_means = trace_lift.posterior['S'].mean(dim=['chain', 'draw']).values
baseline_mean = trace_lift.posterior['baseline'].mean(dim=['chain', 'draw']).values

# Compute lift factors for each channel
channel_lifts = {}
for i, channel in enumerate(media_channels):
    saturated = hill_function(X_media_adstocked[:, i], K_means[i], S_means[i])
    lift = 1 + gamma_means[i] * saturated
    channel_lifts[channel] = lift

# Compute Shapley values
shapley_values = compute_shapley_values(baseline_mean, channel_lifts)

print("\n" + "=" * 60)
print("SHAPLEY VALUE DECOMPOSITION")
print("=" * 60)
print("\nTotal contribution by channel (summed over all weeks):\n")

total_shapley = sum(sv.sum() for sv in shapley_values.values())
for channel in media_channels:
    sv_total = shapley_values[channel].sum()
    sv_pct = (sv_total / total_shapley) * 100 if total_shapley > 0 else 0
    print(f"{channel}:")
    print(f"  Total Shapley contribution: {sv_total:,.0f}")
    print(f"  Share of media effect: {sv_pct:.1f}%")
    print()

In [ ]:
# Visualize Shapley decomposition over time
fig, ax = plt.subplots(figsize=(14, 6))

# Stack the Shapley contributions
bottom = np.full(n_obs, baseline_mean)
colors = plt.cm.Set2(np.linspace(0, 1, n_channels))

for i, channel in enumerate(media_channels):
    ax.fill_between(df['date'], bottom, bottom + shapley_values[channel], 
                    alpha=0.7, label=channel, color=colors[i])
    bottom = bottom + shapley_values[channel]

# Add baseline
ax.axhline(y=baseline_mean, color='black', linestyle='--', label='Baseline', alpha=0.5)

# Add actual sales
ax.plot(df['date'], y_raw, 'k-', linewidth=1.5, label='Actual Sales', alpha=0.7)

ax.set_xlabel('Date')
ax.set_ylabel('Sales')
ax.set_title('Shapley Value Decomposition (Multiplicative Model)')
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()

---
## Part VII: Model Comparison

Compare the Log-Log and Lift-Factor models using:
- Predictive accuracy (MAPE, R-squared)
- Implied ROI/elasticities
- Model fit (LOO-CV)

In [ ]:
# Generate predictions from Log-Log model
with log_log_model:
    ppc_loglog = pm.sample_posterior_predictive(trace_loglog, random_seed=RANDOM_SEED)

# Predictions are in log space, need to transform back
y_pred_loglog_log = ppc_loglog.posterior_predictive['y_obs'].mean(dim=['chain', 'draw']).values
y_pred_loglog = np.exp(y_pred_loglog_log) - 1  # Inverse of log(y+1)

# Metrics for Log-Log
r2_loglog = r2_score(y_raw, y_pred_loglog)
mape_loglog = mean_absolute_percentage_error(y_raw, y_pred_loglog) * 100

print("Log-Log Model Performance:")
print(f"  R-squared: {r2_loglog:.3f}")
print(f"  MAPE: {mape_loglog:.1f}%")

In [ ]:
# Generate predictions from Lift-Factor model
with lift_model:
    ppc_lift = pm.sample_posterior_predictive(trace_lift, random_seed=RANDOM_SEED)

y_pred_lift = ppc_lift.posterior_predictive['y_obs'].mean(dim=['chain', 'draw']).values

# Metrics for Lift-Factor
r2_lift = r2_score(y_raw, y_pred_lift)
mape_lift = mean_absolute_percentage_error(y_raw, y_pred_lift) * 100

print("Lift-Factor Model Performance:")
print(f"  R-squared: {r2_lift:.3f}")
print(f"  MAPE: {mape_lift:.1f}%")

In [ ]:
# Visual comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time series: Log-Log
ax = axes[0, 0]
ax.plot(df['date'], y_raw, 'b-', label='Actual', alpha=0.7)
ax.plot(df['date'], y_pred_loglog, 'r--', label='Log-Log Predicted', alpha=0.7)
ax.set_title(f'Log-Log Model (R2={r2_loglog:.3f}, MAPE={mape_loglog:.1f}%)')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Sales')

# Time series: Lift-Factor
ax = axes[0, 1]
ax.plot(df['date'], y_raw, 'b-', label='Actual', alpha=0.7)
ax.plot(df['date'], y_pred_lift, 'g--', label='Lift Predicted', alpha=0.7)
ax.set_title(f'Lift-Factor Model (R2={r2_lift:.3f}, MAPE={mape_lift:.1f}%)')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Sales')

# Scatter: Log-Log
ax = axes[1, 0]
ax.scatter(y_raw, y_pred_loglog, alpha=0.5)
ax.plot([y_raw.min(), y_raw.max()], [y_raw.min(), y_raw.max()], 'r--')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title('Log-Log: Actual vs Predicted')

# Scatter: Lift-Factor
ax = axes[1, 1]
ax.scatter(y_raw, y_pred_lift, alpha=0.5, color='green')
ax.plot([y_raw.min(), y_raw.max()], [y_raw.min(), y_raw.max()], 'r--')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title('Lift-Factor: Actual vs Predicted')

plt.tight_layout()
plt.show()

In [ ]:
# Summary comparison table
print("\n" + "=" * 70)
print("MODEL COMPARISON SUMMARY")
print("=" * 70)

comparison_data = {
    'Metric': ['R-squared', 'MAPE (%)', 'Coefficient Type', 'Handles Zeros', 'Natural Interactions'],
    'Log-Log': [f'{r2_loglog:.3f}', f'{mape_loglog:.1f}', 'Elasticities', 'Yes (log1p)', 'No'],
    'Lift-Factor': [f'{r2_lift:.3f}', f'{mape_lift:.1f}', 'Lift %', 'Yes', 'Yes']
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

---
## Part VIII: ROI Calculation for Multiplicative Models

ROI calculation differs between model types:

- **Log-Log**: ROI = (elasticity * average_sales) / average_spend
- **Lift-Factor**: Use Shapley values for contribution, then ROI = contribution / spend

In [ ]:
# ROI from Shapley values (Lift-Factor model)
print("\n" + "=" * 60)
print("ROI CALCULATION (Lift-Factor Model via Shapley)")
print("=" * 60)

roi_shapley = {}
for i, channel in enumerate(media_channels):
    total_spend = df[channel].sum()
    total_contribution = shapley_values[channel].sum()
    
    if total_spend > 0:
        roi = total_contribution / total_spend
    else:
        roi = 0
    
    roi_shapley[channel] = roi
    print(f"\n{channel}:")
    print(f"  Total Spend: ${total_spend:,.0f}")
    print(f"  Shapley Contribution: {total_contribution:,.0f} sales")
    print(f"  ROI: {roi:.2f}x")

In [ ]:
# ROI from elasticities (Log-Log model)
print("\n" + "=" * 60)
print("ROI CALCULATION (Log-Log Model via Elasticities)")
print("=" * 60)
print("\nNote: Elasticity-based ROI is approximate\n")

avg_sales = y_raw.mean()
roi_elasticity = {}

for i, channel in enumerate(media_channels):
    elasticity = elasticities[i]
    avg_spend = df[channel].mean()
    
    if avg_spend > 0:
        # ROI approximation: (elasticity * avg_sales) / avg_spend
        # This gives: d(Sales)/d(Spend) at the average
        roi = (elasticity * avg_sales) / avg_spend
    else:
        roi = 0
    
    roi_elasticity[channel] = roi
    print(f"{channel}:")
    print(f"  Elasticity: {elasticity:.4f}")
    print(f"  Avg Spend: ${avg_spend:,.0f}")
    print(f"  Approximate ROI: {roi:.2f}x")
    print()

In [ ]:
# Visual ROI comparison
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(media_channels))
width = 0.35

bars1 = ax.bar(x - width/2, [roi_elasticity.get(ch, 0) for ch in media_channels], 
               width, label='Log-Log (Elasticity)', color='steelblue')
bars2 = ax.bar(x + width/2, [roi_shapley.get(ch, 0) for ch in media_channels], 
               width, label='Lift-Factor (Shapley)', color='seagreen')

ax.set_xlabel('Channel')
ax.set_ylabel('ROI')
ax.set_title('ROI Comparison: Log-Log vs Lift-Factor Models')
ax.set_xticks(x)
ax.set_xticklabels([ch.replace('_SPEND', '') for ch in media_channels], rotation=45, ha='right')
ax.legend()
ax.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='Break-even')

plt.tight_layout()
plt.show()

---
## Part IX: Budget Optimization (Multiplicative Models)

Budget optimization for multiplicative models uses the same principle as additive:
**Equalize marginal ROI across channels.**

However, the calculation differs:

- **Log-Log**: Marginal effect = elasticity * (Y / X) at current levels
- **Lift-Factor**: Use Shapley-based marginal contributions

In [ ]:
def calculate_marginal_roi_loglog(spend, elasticity, avg_sales, avg_spend):
    """
    Calculate marginal ROI for log-log model.
    
    For log-log: d(Sales)/d(Spend) = elasticity * (Sales / Spend)
    
    At a given spend level, marginal ROI decreases as spend increases
    (because Sales/Spend ratio changes).
    """
    # Approximate sales at this spend level using elasticity
    # log(Y) = log(Y_avg) + elasticity * log(Spend / Spend_avg)
    sales_at_spend = avg_sales * (spend / avg_spend) ** elasticity
    
    # Marginal effect: d(Sales)/d(Spend) = elasticity * Sales / Spend
    marginal_sales = elasticity * sales_at_spend / spend if spend > 0 else 0
    
    return marginal_sales


def calculate_marginal_roi_lift(spend, gamma, K, S, baseline, avg_spend):
    """
    Calculate marginal ROI for lift-factor model.
    
    Lift = 1 + gamma * Hill(spend)
    Marginal = baseline * gamma * Hill'(spend)
    """
    # Scale spend
    x = spend / avg_spend if avg_spend > 0 else spend
    
    # Hill derivative: d/dx [x^S / (K^S + x^S)] = S * K^S * x^(S-1) / (K^S + x^S)^2
    if x > 0:
        numerator = S * (K ** S) * (x ** (S - 1))
        denominator = (K ** S + x ** S) ** 2 + 1e-10
        hill_deriv = numerator / denominator
    else:
        hill_deriv = 0
    
    # Marginal contribution (in sales units)
    marginal = baseline * gamma * hill_deriv / avg_spend if avg_spend > 0 else 0
    
    return marginal

print("Marginal ROI functions defined.")

In [ ]:
# Calculate current marginal ROI for each channel (Log-Log model)
print("=" * 60)
print("CURRENT MARGINAL ROI (Log-Log Model)")
print("=" * 60)
print("\nMarginal ROI = return on the NEXT dollar spent\n")

avg_sales = y_raw.mean()
marginal_rois_loglog = {}

for i, channel in enumerate(media_channels):
    avg_spend = df[channel].mean()
    current_spend = avg_spend  # Evaluate at current spend level
    
    mroi = calculate_marginal_roi_loglog(
        spend=current_spend,
        elasticity=elasticities[i],
        avg_sales=avg_sales,
        avg_spend=avg_spend
    )
    
    marginal_rois_loglog[channel] = mroi
    print(f"{channel.replace('_SPEND', '')}:")
    print(f"  Current weekly spend: ${current_spend:,.0f}")
    print(f"  Elasticity: {elasticities[i]:.4f}")
    print(f"  Marginal ROI: {mroi:.4f}")
    print()

In [ ]:
from scipy.optimize import minimize

def optimize_budget_loglog(total_budget, channels, elasticities_dict, 
                           avg_sales, avg_spends, min_pct=0.1, max_pct=0.8):
    """
    Optimize budget allocation for log-log model.
    
    Maximizes total sales given budget constraint.
    """
    n = len(channels)
    
    def neg_total_sales(allocation):
        """Negative total sales (for minimization)."""
        total = 0
        for i, ch in enumerate(channels):
            spend = allocation[i]
            el = elasticities_dict[ch]
            avg_sp = avg_spends[ch]
            # Sales contribution: avg_sales * (spend/avg_spend)^elasticity
            if spend > 0 and avg_sp > 0:
                contribution = avg_sales * (spend / avg_sp) ** el
            else:
                contribution = 0
            total += contribution
        return -total
    
    # Constraints: sum of allocation = total budget
    constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - total_budget}]
    
    # Bounds: min and max spend per channel
    bounds = [(total_budget * min_pct / n, total_budget * max_pct) for _ in range(n)]
    
    # Initial guess: equal split
    x0 = np.array([total_budget / n] * n)
    
    # Optimize
    result = minimize(neg_total_sales, x0, method='SLSQP',
                      bounds=bounds, constraints=constraints)
    
    if result.success:
        return {
            'success': True,
            'allocation': {ch: result.x[i] for i, ch in enumerate(channels)},
            'total_sales': -result.fun
        }
    else:
        return {'success': False, 'message': result.message}

print("Budget optimization function defined.")

In [ ]:
# Run budget optimization
print("=" * 60)
print("BUDGET OPTIMIZATION (Log-Log Model)")
print("=" * 60)

# Prepare inputs
elasticities_dict = {ch: elasticities[i] for i, ch in enumerate(media_channels)}
avg_spends = {ch: df[ch].mean() for ch in media_channels}
current_budget = sum(avg_spends.values())

print(f"\nCurrent total weekly budget: ${current_budget:,.0f}")

# Optimize
opt_result = optimize_budget_loglog(
    total_budget=current_budget,
    channels=media_channels,
    elasticities_dict=elasticities_dict,
    avg_sales=avg_sales,
    avg_spends=avg_spends,
    min_pct=0.05,
    max_pct=0.9
)

if opt_result['success']:
    print("\nOptimization successful!")
    print("\nCurrent vs Optimal Allocation:")
    print("-" * 50)
    
    comparison_data = []
    for ch in media_channels:
        current = avg_spends[ch]
        optimal = opt_result['allocation'][ch]
        change_pct = 100 * (optimal - current) / current if current > 0 else 0
        comparison_data.append({
            'Channel': ch.replace('_SPEND', ''),
            'Current': current,
            'Optimal': optimal,
            'Change (%)': change_pct
        })
        print(f"{ch.replace('_SPEND', '')}:")
        print(f"  Current: ${current:,.0f}")
        print(f"  Optimal: ${optimal:,.0f}")
        print(f"  Change: {change_pct:+.1f}%")
        print()
    
    comparison_df = pd.DataFrame(comparison_data)
else:
    print(f"\nOptimization failed: {opt_result.get('message', 'Unknown error')}")
    comparison_df = None

In [ ]:
# Visualize current vs optimal allocation
if opt_result['success']:
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    channels_display = [c.replace('_SPEND', '') for c in media_channels]
    current_values = [avg_spends[ch] for ch in media_channels]
    optimal_values = [opt_result['allocation'][ch] for ch in media_channels]
    
    # Plot 1: Bar chart - Current vs Optimal
    ax = axes[0]
    x = np.arange(len(channels_display))
    width = 0.35
    bars1 = ax.bar(x - width/2, current_values, width, label='Current', color='steelblue')
    bars2 = ax.bar(x + width/2, optimal_values, width, label='Optimal', color='coral')
    ax.set_xlabel('Channel')
    ax.set_ylabel('Weekly Spend ($)')
    ax.set_title('Current vs Optimal Budget Allocation')
    ax.set_xticks(x)
    ax.set_xticklabels(channels_display, rotation=45, ha='right')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # Plot 2: Change percentage
    ax = axes[1]
    changes = comparison_df['Change (%)'].values
    colors = ['green' if c > 0 else 'red' for c in changes]
    ax.barh(channels_display, changes, color=colors, alpha=0.7)
    ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    ax.set_xlabel('Change (%)')
    ax.set_title('Recommended Budget Changes')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Plot 3: Pie charts
    ax = axes[2]
    # Create side-by-side pie charts using subplots
    ax.axis('off')
    
    # Create inset axes for pie charts
    ax1 = fig.add_axes([0.68, 0.15, 0.15, 0.7])  # Current
    ax2 = fig.add_axes([0.84, 0.15, 0.15, 0.7])  # Optimal
    
    ax1.pie(current_values, labels=channels_display, autopct='%1.0f%%', 
            colors=plt.cm.Set2(np.linspace(0, 1, len(channels_display))))
    ax1.set_title('Current', fontsize=10)
    
    ax2.pie(optimal_values, labels=channels_display, autopct='%1.0f%%',
            colors=plt.cm.Set2(np.linspace(0, 1, len(channels_display))))
    ax2.set_title('Optimal', fontsize=10)
    
    plt.tight_layout()
    plt.show()
else:
    print("Cannot visualize - optimization did not succeed.")

In [ ]:
# Calculate expected improvement
if opt_result['success']:
    print("=" * 60)
    print("EXPECTED IMPROVEMENT")
    print("=" * 60)
    
    # Calculate current expected sales
    current_sales = 0
    for i, ch in enumerate(media_channels):
        spend = avg_spends[ch]
        el = elasticities[i]
        contribution = avg_sales * (spend / avg_spends[ch]) ** el if spend > 0 else 0
        current_sales += contribution
    
    # Optimal sales from optimization
    optimal_sales = opt_result['total_sales']
    
    improvement = (optimal_sales - current_sales) / current_sales * 100 if current_sales > 0 else 0
    
    print(f"\nCurrent expected media-driven sales: {current_sales:,.0f}")
    print(f"Optimal expected media-driven sales: {optimal_sales:,.0f}")
    print(f"Expected improvement: {improvement:+.1f}%")
    
    # Marginal ROI at optimal allocation
    print("\nMarginal ROI at Optimal Allocation:")
    print("-" * 40)
    for i, ch in enumerate(media_channels):
        opt_spend = opt_result['allocation'][ch]
        mroi = calculate_marginal_roi_loglog(
            spend=opt_spend,
            elasticity=elasticities[i],
            avg_sales=avg_sales,
            avg_spend=avg_spends[ch]
        )
        print(f"  {ch.replace('_SPEND', '')}: {mroi:.4f}")
    
    print("\n(At optimum, marginal ROIs should be approximately equal)")

---
## Part X: Summary and Recommendations

### Key Takeaways

1. **Log-Log Model**:
   - Coefficients are elasticities (% change in sales per % change in spend)
   - Simpler to implement and interpret
   - Good when interactions are not the primary concern

2. **Lift-Factor Model**:
   - Naturally captures channel interactions through multiplication
   - Requires Shapley values for fair decomposition
   - Better when synergies between channels are important

3. **Shapley Values**:
   - Essential for fair attribution in multiplicative models
   - Computationally expensive (2^n coalitions)
   - Satisfies key fairness axioms

### When to Choose Each Model

| Scenario | Recommended Model |
|----------|------------------|
| Simple reporting, stakeholder communication | Additive |
| Strong channel interactions expected | Multiplicative (Lift) |
| Quick elasticity estimates needed | Log-Log |
| Using tools like Robyn, PyMC-Marketing | Additive (default) |
| Using Google Meridian | Multiplicative |

In [ ]:
# Final summary
print("=" * 70)
print("MULTIPLICATIVE MMM - EXECUTIVE SUMMARY")
print("=" * 70)

print("\n--- LOG-LOG MODEL ---")
print(f"Model Fit: R2 = {r2_loglog:.3f}, MAPE = {mape_loglog:.1f}%")
print("\nElasticities (1% spend increase -> X% sales increase):")
for i, ch in enumerate(media_channels):
    print(f"  {ch.replace('_SPEND', '')}: {elasticities[i]:.4f}")

print("\n--- LIFT-FACTOR MODEL ---")
print(f"Model Fit: R2 = {r2_lift:.3f}, MAPE = {mape_lift:.1f}%")
print(f"Baseline Sales: {baseline_mean:,.0f}/week")
print("\nMaximum Lift Potential:")
for i, ch in enumerate(media_channels):
    print(f"  {ch.replace('_SPEND', '')}: +{gamma_means[i]*100:.1f}%")

print("\n--- SHAPLEY ROI ---")
for ch in media_channels:
    print(f"  {ch.replace('_SPEND', '')}: {roi_shapley[ch]:.2f}x")

print("\n" + "=" * 70)

---
## Next Steps

1. **Compare with Additive Model**: Run the main notebook and compare ROI estimates
2. **Validate with Experiments**: Test elasticity/lift estimates against geo-lift results
3. **Sensitivity Analysis**: Test how results change with different priors
4. **Time-Varying Effects**: Consider if elasticities change over time

---

*This notebook implements concepts from Chapter 9b and Appendix G of the MMM Complete Guide.*